In [1]:
from bs4 import BeautifulSoup as bs

import requests
import re


import pandas as pd
import numpy as np



import time
import random

In [13]:
# this is grabing the Cardinals html web page
url = 'https://www.pro-football-reference.com/teams/was/2022.htm'

In [3]:
#Table Function

In [4]:
def tables_html(url:str):
    html = requests.get(url).text
    
    soup = bs(html, 'html.parser')
    
    tables = soup.find_all(class_='table_wrapper')
    
    return tables

# DF Columns!

In [5]:
def webscrape_columns (tables:str, table_num:int):

    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')
    
    
    #find thead tag to isolate col names
    table_html = table_html.find_all('thead')[0]

    # find class_ = 'poptip' to isolate col names
    col_th_tag = table_html.find_all('th')

    # get how many column names there are
    num_col_passing = len(col_th_tag)

    # list of column names
    columns = []

    for i in range(0, num_col_passing):
        try:
            data = col_th_tag[i].contents[0]
            columns.append(data)
        except:
            columns.append('N/A')   
            
    return columns

In [14]:
tables = tables_html(url=url)

In [16]:
col=webscrape_columns(tables=tables, table_num=1)
col

['N/A',
 'Score',
 'Offense',
 'Defense',
 'Expected Points',
 'Week',
 'Day',
 'Date',
 'N/A',
 'N/A',
 'N/A',
 'OT',
 'Rec',
 'N/A',
 'Opp',
 'Tm',
 'Opp',
 '1stD',
 'TotYd',
 'PassY',
 'RushY',
 'TO',
 '1stD',
 'TotYd',
 'PassY',
 'RushY',
 'TO',
 'Offense',
 'Defense',
 'Sp. Tms']

In [20]:
pd.DataFrame(columns=col)

,N/A,Score,Offense,Defense,Expected Points,Week,Day,Date,N/A,N/A,...,RushY,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms


# DF Rows!

## Row

In [6]:
def webscrape_index(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
       
    #get index
    num_index = len(rows_tbody.find_all('th'))
    
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
    return index

In [7]:
def webscrape_rows(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
    

    rows_tr = rows_tbody.find_all('tr')
    
    num_index = len(rows_tbody.find_all('th'))
    num_rows = len(rows_tr[0].find_all('td'))
    
    
    row_data = []

    for i in range(0, num_index):
        row_td = rows_tr[i].find_all('td')
    
        data = []

        for c in range(0, num_rows):
            
            try:
                if len(row_td[c].find_all('a')) > 0:
            
                    d = row_td[c].find_all('a')[0].contents[0]
                    data.append(d)
                
                else:
                    d = row_td[c].contents[0]
                    data.append(d)
            
            except:
                data.append('n/a')
        

        row_data.append(data)
        
        
        
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
        
    #insert index to each row to be equal to all columns
    for i in range(0, len(index)):
        row_data[i].insert(0, index[i])
        
        
    return row_data

# Fix Duplicate Columns

In [8]:
def fix_col_labels(tables_li: list ):
    
    #Team Stats and Rankings
    col_li = list(tables_li[0].columns)
    
    col_li[2] = 'Total Yds and TO'
    col_li[10] = 'Passing Yds'
    col_li[16] = 'Russhing Yds'
    col_li[21] = 'Penalty Yds'

    tables_li[0].columns = col_li
    
    
    #Passing
    col_li = list(tables_li[3].columns)
    
    col_li[10] = 'Yds Gained'
    col_li[20] = 'Yds Lost'

    tables_li[3].columns = col_li
    
    
    #Rushing and Receiving
    col_li = list(tables_li[4].columns)
    
    col_li[7] = 'Games Yds'
    col_li[17] = 'Rushing Yds'

    tables_li[4].columns = col_li
    
    #kick and punt returns
    col_li = list(tables_li[5].columns)
    
    col_li[7] = 'Punt Returns Yds'
    col_li[12] = 'Kick Returns Yds'

    tables_li[5].columns = col_li
    
    #Kicking
    col_li = list(tables_li[6].columns)
    
    #0-19
    col_li[6] = 'FGA 0-19'
    col_li[7] = 'FGM 0-19'
    
    #20-29
    col_li[8] = 'FGA 20-29'
    col_li[9] = 'FGM 20-29'
    
    #30-39
    col_li[10] = 'FGA 30-39'
    col_li[11] = 'FGM 30-39'
    
    #40-49
    col_li[12] = 'FGA 40-49'
    col_li[13] = 'FGM 40-49'
    
    #50+
    col_li[14] = 'FGA 50+'
    col_li[15] = 'FGM 50+'

    tables_li[6].columns = col_li
    
    #defense and fumbles
    col_li = list(tables_li[9].columns)
    
    col_li[7] = 'Def Interceptions Yds'
    col_li[14] = 'Fumbles Yds'

    tables_li[9].columns = col_li
    
    return tables_li

# DF!

In [9]:
#df = pd.DataFrame(index=index, data=row_data, columns=columns)
#df

In [10]:
def webscrape_tables(tables:str, table_num:int):
        
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
    #columns
    #find thead tag to isolate col names
    table_html = table_html.find_all('thead')[0]

    # find th to isolate col names
    col_th_tag = table_html.find_all('th')

    # get how many column names there are
    num_col_passing = len(col_th_tag)

    # list of column names
    columns = []

    for i in range(0, num_col_passing):
        try:
            data = col_th_tag[i].contents[0]
            columns.append(data)
        except:
            columns.append('N/A') 
    
    ##rows
    rows_tr = rows_tbody.find_all('tr')
    
    num_index = len(rows_tbody.find_all('th'))
    num_rows = len(rows_tr[0].find_all('td'))
    
    
    row_data = []

    for i in range(0, num_index):
        row_td = rows_tr[i].find_all('td')
    
        data = []

        for c in range(0, num_rows):
            
            try:
                if len(row_td[c].find_all('a')) > 0:
            
                    d = row_td[c].find_all('a')[0].contents[0]
                    data.append(d)
                
                else:
                    d = row_td[c].contents[0]
                    data.append(d)
            
            except:
                data.append('n/a')
        

        row_data.append(data)
        
        
        
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
        
    #insert index to each row to be equal to all columns
    for i in range(0, len(index)):
        row_data[i].insert(0, index[i])
            
    
       
    #get index
    num_index = len(rows_tbody.find_all('th'))
    
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')       
    
    
    
    df = pd.DataFrame(index=index, data=row_data, columns=columns)
    
    return df

In [147]:
tables

[]

In [12]:
webscrape_tables(tables=tables, table_num=1)

ValueError: 30 columns passed, passed data had 25 columns